In [22]:
import sys  
sys.path.insert(0, '..')

from datetime import date, timedelta

from config import Config

import pandas as pd
from sqlalchemy import create_engine

db = create_engine(Config.SQLALCHEMY_DATABASE_URI)

In [23]:
ockovani = pd.read_sql_query(
    """
    select l.orp_bydl_kod, 1.0 * sum(l.pocet) / min(p.pocet) pocet_ockovani
    from ockovani_lide l
    join obce_orp o on o.uzis_orp = l.orp_bydl_kod
    join populace_orp p on p.orp_kod = o.kod_obce_orp
    where poradi_davky = 1
    group by orp_bydl_kod
    """,
    db
)
ockovani

,orp_bydl_kod,pocet_ockovani
0,1000,0.579946
1,2001,0.613386
2,2002,0.590540
3,2003,0.570634
4,2004,0.613178
...,...,...
201,8018,0.514378
202,8019,0.536991
203,8020,0.549566
204,8021,0.482409


In [24]:
nakazeni = pd.read_sql_query(
    """
    select c.orp_kod, 1.0 * c.aktivni_pripady / p.pocet aktivni_pripady
    from charakteristika_obci c
    join obce_orp o on o.uzis_orp = c.orp_kod
    join populace_orp p on p.orp_kod = o.kod_obce_orp
    where datum = '2021-10-14'
    """,
    db
)
nakazeni

,orp_kod,aktivni_pripady
0,1000,0.001718
1,2001,0.001011
2,2025,0.000771
3,2026,0.000241
4,2002,0.002073
...,...,...
203,8009,0.004919
204,8013,0.001221
205,8017,0.002456
206,8022,0.003853


In [25]:
df = pd.merge(ockovani, nakazeni, left_on='orp_bydl_kod', right_on='orp_kod')

In [26]:
df[['pocet_ockovani', 'aktivni_pripady']].corr()

,pocet_ockovani,aktivni_pripady
pocet_ockovani,1.000000,-0.098978
aktivni_pripady,-0.098978,1.000000
